# Video Creation Script

In [30]:
# Import all required modules
import pandas as pd
import random
from pytube import YouTube
from moviepy.editor import *

### Video and Audio Selection

In [31]:
# Import the videos from working directory
audio_options = pd.read_csv("NoCopyrightSounds - WithDescriptions.csv")
vids = pd.read_csv("Background Video Links.csv")

In [32]:
# Choose a random audio file and pull its youtube URL
selected_audio = audio_options.sample()
audio_url = selected_audio['videoURL'].iloc[0]

In [33]:
# Here we aim to get the YouTube objects and video lengths of all background
# video options.
vid_length = []
video_yt = []
vid_time = 0

# A loop is used because sometime YouTube function fails to connect to the video
# so if we set length = 0 when this occurs we can loop videos until the object is
# returned. Once we have the object we have all information about the video.
for vid in vids['video_url']:
    while vid_time == 0:
        vid_object = YouTube(vid)
        try:
            vid_time = vid_object.length
        except:
            vid_time = 0
    
    # Once we have connection then append the time and the object itself
    vid_length.append(vid_time)
    video_yt.append(vid_object)
    # Set the vid_time back to zero below to begin looking at next video.
    vid_time = 0 
        
vid_length = pd.DataFrame({'video_length':vid_length})

In [34]:
# Create a dataframe out of the video information and extract the video id
vid_data = pd.DataFrame({'video_url':vids['video_url'], 'video_yt':video_yt})
vid_data['video_id'] = vid_data['video_url'].str.extract(r'v=(\w+)')
vid_data = pd.concat([vid_data, vid_length], axis=1)

In [35]:
# Want to return length of audio, same as for the video files
audio_length = 0
while audio_length == 0:
        audio_yt = YouTube(audio_url)
        try:
            audio_length = audio_yt.length
        except:
            audio_length = 0

In [36]:
# Filter videos for only videos longer than the selected audio file
video_options = vid_data[vid_data["video_length"] > audio_length] 

# Take a sample video
selected_video = video_options.sample()['video_yt'].iloc[0]

In [37]:
# selected_video.watch_url

In [38]:
# audio_yt.watch_url

### Video and Audio Download

In [39]:
# Return the start and end points for background video. Want video to start
# at any point between 0s and what will get full audio file included
start_point = random.randint(0, selected_video.length - audio_yt.length -1)
end_point = start_point + audio_yt.length

In [40]:
# Download the audio and video files to audio.mp3 and video_no_audio.mp4 in working directory
audio_yt.streams.filter(only_audio = True, file_extension = 'mp4').first().download(filename="audio.mp3")
selected_video.streams.filter(only_video = True, file_extension = 'mp4').first().download(filename = "video_no_audio.mp4")

'C:\\Users\\bencros\\Documents\\GitHub\\TikTokSideHustle\\video_no_audio.mp4'

### Video Creation

In [41]:
# Read the audio and video files
video_clip = VideoFileClip("video_no_audio.mp4")
audio_clip = AudioFileClip("audio.mp3")

# Create a video from chosen start and endpoints
# video_clip = video_clip.subclip(0, audio_length) - KEEPING IN CASE I REVERT BACK
video_clip = video_clip.subclip(start_point, end_point)

# Code provided to make audio work on video
new_audioclip = CompositeAudioClip([audio_clip])
video_clip.audio = new_audioclip

# Write the video to an mp4 file
video_clip.write_videofile("video.mp4")

Moviepy - Building video video.mp4.
MoviePy - Writing audio in videoTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video video.mp4



Moviepy - Done !
Moviepy - video ready video.mp4
